# Final Project Notebook
Group: 9
Group Members: Shawn Ericksen (ericksen@uark.edu), Garret Fulghum (gmfulghu@uark.edu), Wesley Parker (wgparker@uark.edu)

This practice project focuses on the Microsoft Malware Prediction dataset. This can be accessed from: https://www.kaggle.com/competitions/microsoft-malware-prediction/data

Below is all of our specified imports

In [ ]:
import numpy as np
import pandas as pd
import csv

# from scipy import sparse

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import CategoricalNB
from sklearn.svm import SVC

In [ ]:
def clean(df):
#     drop_cols = []
    for col in df.columns:
        if df[col].dtype == 'float16' or df[col].dtype == 'float32' or df[col].dtype == 'float64':
            df[col] = (df[col] >= df.mean(numeric_only=True)[col]).astype(int)
            
#     df.drop(labels=cols, axis=1)
    
    return df

In [ ]:
dtypes={'id': 'string', 'date_account_created': 'string', 'timestamp_first_active': 'string', 'date_first_booking': 'string', 'gender': 'category', 'age': 'Int64', 'signup_method': 'category', 'signup_flow': 'category', 'language': 'category', 'affiliate_channel': 'category', 'affiliate_provider': 'category', 'first_affiliate_tracked': 'category', 'signup_app': 'category', 'first_device_type': 'category', 'first_browser': 'category', 'country_destination': 'category'}
parse_dates = ['date_account_created', 'timestamp_first_active', 'date_first_booking']

In [ ]:
filepath = "airbnb-recruiting-new-user-bookings/train_users_2.csv"
cols = list(pd.read_csv(filepath, nrows=1))[1:]
df = pd.read_csv(filepath, dtype=dtypes, na_values=['-unknown-', 'NDF', '<NA>'], usecols =[i for i in cols if i != 'id'], parse_dates=parse_dates, infer_datetime_format=True)
df['date_first_booking'] = pd.to_datetime(pd.Series(df['date_first_booking']), format='%Y-%m-%d', errors='coerce')
for i in list(df):
    if df.dtypes[i] == 'datetime64[ns]':
        df[i].fillna(pd.NaT)

In [ ]:
print("- - Adult.Data - -")
print("Number of lines present: ", len(df))
print("Number of Columns: ", len(df.columns))

In [ ]:
topCount = 5
print("Top ", topCount, " dataFrames:")
print(df.head(topCount))

In [ ]:
# df.memory_usage(deep=True, index=False).sort_values(ascending=False)

In [ ]:
df.memory_usage(deep=True, index=False).sum()

In [ ]:
# coo = sparse.coo_matrix(df_train)
# csr = coo.tocsr()
# print(csr)

## This section is for developing the codebase since true lables for test.csv are not given.

In [ ]:
X = df.drop(columns=['country_destination'])
y = df['country_destination']

In [ ]:
# This block can be commented out when doing prediction on the Kaggle test.csv
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.37, random_state=0, shuffle=True)

In [ ]:
cols = [i for i in range(0, len(list(df))-1) if df.dtypes[i] == 'category']
cols

In [ ]:
# encoder = ColumnTransformer([('one_hot_encoder', OneHotEncoder(categories='auto'), cols)], remainder='passthrough')
encoder = make_column_transformer((OneHotEncoder(sparse=False, handle_unknown='ignore'), make_column_selector(dtype_include='category')))
X_train = encoder.fit_transform(X_train)
y_enc = LabelEncoder().fit(y_train.values)
y_train = y_enc.transform(y_train)
X_test = encoder.transform(X_test)
y_test = y_enc.transform(y_test.values)

### Classifiers

In [ ]:
clf1 = CategoricalNB()
clf1.fit(X_train, y_train)

In [ ]:
y_pred = clf1.predict(X_test)
print(classification_report(y_test, y_pred, target_names=X.columns[:-2]))

In [ ]:
clf2 = SVC().fit(X_train, y_train)

In [ ]:
y_pred = clf2.predict(X_test)
print(classification_report(y_test, y_pred, target_names=X.columns[:-2]))

In [ ]:
# clf3 = KMeans(n_clusters=3, max_iter=100, random_state=0)
# clf3.fit(X_train, y_train)

## Predictions for Kaggle's test.csv

In [ ]:
# filepath = "test.csv"
# cols = list(pd.read_csv(filepath, nrows=1))
# ddf = dd.read_csv(filepath, dtype=dtypes, blocksize="16MB", na_values=['NA', 'UNKNOWN', 'NOT_SET', 'nan'])

In [ ]:
# ddf_test = ddf.categorize()

In [ ]:
# id_list = list(ddf['MachineIdentifier'])
# X_kaggle = encoder.transform(ddf.drop(columns=['MachineIdentifier']))

In [ ]:
# y_pred = clf1.predict(X_kaggle)

In [ ]:
# with open("submission.csv", "w", newline="") as csvfile:
#     csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"')
#     csv_writer.writerows(zip(id_list, y_pred))